In [ ]:
from tkinter import filedialog
from tkinter import *
root = Tk()
root.withdraw()

import os
import openpyxl
import pandas as pd
import time

# Input

In [ ]:
filepath_topics = filedialog.askopenfilename(title="Select file listing Topics (projects)")
print(filepath_topics)

sheet_name = 'Topics'

try:
    df_topics = pd.read_excel(filepath_topics, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

In [ ]:
filepath_students = filedialog.askopenfilename(title="Select file listing Students and their preferences")
print(filepath_students)

# Read 'Students' sheet
sheet_name = 'Students'

try:
    df_students = pd.read_excel(filepath_students, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")


# Read 'Students preferences' sheet
sheet_name = 'Student_preferences'

try:
    df_students_preferences = pd.read_excel(filepath_students, sheet_name=sheet_name)
    #df_preferences = pd.read_excel(filepath_student_preferences, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

df_students_preferences.rename(columns={'Your name (Lastname, First name)' : 'TypedName',
                        'Have you submitted a Thesis Self-Proposal?': 'Self-proposal',
                        'Do you wish to work in a pair': 'Pair',
                        'Please enter the name of your partner': 'Partner',
                        'Please select the project code of your 1st preferred research Topic': 'Topic1',
                        'Please select the project code of your 2nd preferred research Topic': 'Topic2',
                        'Please select the project code of your 3rd preferred research Topic': 'Topic3',
                        'Have you had a meaningful discussion with the supervisors of those first three research Topics you have selected (by email, Teams or in-person)?': 'Discussion',
                        'Please select the project code of your 4th preferred research Topic': 'Topic4', 
                        'Please select the project code of your 5th preferred research Topic': 'Topic5',
                        'Please, select your 1st preferred Subject Area that you would be happy to work in (see Thesis Handbook for details).' : 'SubjectArea1',
                        'Please, select your 2nd preferred Subject Area that you would be happy to work in.' : 'SubjectArea2',
                        'Please, select your 3rd preferred Subject Area that you would be happy to work in.' : 'SubjectArea3',
                        'Please, select your 1st preferred Research Type that you would be happy to work on.' : 'ProjectType1',
                        'Please, select your 2nd preferred Research Type that you would be happy to work on.' : 'ProjectType2'},
                         inplace = True)

print(df_students_preferences.columns) 


# Read 'Students self_proposals' sheet
sheet_name = 'Student_selfproposals'

try:
    df_selfproposals = pd.read_excel(filepath_students, sheet_name=sheet_name)
    print(f"{sheet_name} data loaded successfully!")
except Exception as e:
    print(f"An error occurred while loading {sheet_name} data: {e}")

In [ ]:
filepath_supervisors = filedialog.askopenfilename(title="Select file listing Supervisors")
print(filepath_supervisors)

sheet_name = 'Supervisors'
try:
    df_supervisors = pd.read_excel(filepath_supervisors, sheet_name=sheet_name)
    print("'Supervisors' Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
filepath_final_allocations = filedialog.askopenfilename(title="Select file listing the Finale Allocations ()")
print(filepath_final_allocations)

sheet_name = 'Final Allocations'
try:
    df_results = pd.read_excel(filepath_final_allocations, sheet_name=sheet_name)
    print("'Final Allocations' Data loaded successfully!")
except Exception as e:
    print(f"An error occurred: {e}")


# Create Spreadsheet for Internal Use

In [ ]:
# ---------------------------------------------
# Create filepath for the results 
# --------------------------------------------- 
folder_results = os.path.dirname(os.path.abspath(filepath_allocations))

timestr = time.strftime("%Y%m%d-%H%M%S")

file_internal_results = "04_Output_" + timestr + ".xlsx"
filepath_internal_results = os.path.join(folder_results,file_internal_results)

wb = openpyxl.Workbook()
wb.save(filepath_internal_results)


# -------------------------------
# CREATE STUDENT ALLOCATION SHEET 
# -------------------------------

# Create new DataFrame to save final allocations as table
# -------------------------------

# Define the column names
internal_results_columns = ['Student', 'Email', 'Code', 'Supervisor', 'Title', 'Subject Area', 'Type', 'Source', 'Rank']

print(internal_results_columns)

# Create an empty DataFrame with specified columns
df_internal_results = pd.DataFrame(columns=internal_results_columns)


# Add data to results DataFrame
for index, row in df_results.iterrows():

    student_name = row['Student']
    student_email = row['Email']
    project_code = row['Code']
    project_supervisor = row['Supervisor']
    project_title = row['Title']
    project_area = row['Subject Area']
    project_type = row['Type']
    project_source = row['Source']
    
    
    '''
    student_allocation_rows = df_student_allocations.loc[df_student_allocations['Student'] == student_name] 

    if student_allocation_rows.empty == False:
        
        #print("student_allocation_rows: ", student_allocation_rows)
        
        for index2, row2 in student_allocation_rows.iterrows():
            
            #print("student allocation row: ", row2)   
            student_rank = int(row2['Student_preference_rank'])
    '''
    student_rank = 1
    
    if project_source == "Directory":
        row = df_students_preferences.loc[df_students_preferences['Email'] == student_email].iloc[0]
        
        student_topics = [row['Topic1'], row['Topic2'], row['Topic3'], row['Topic4'], row['Topic5']]
        
        student_rank = student_topics.index(project_code) + 1

    student_internal_result = pd.DataFrame({'Student' : [student_name],
                                   'Email' : [student_email],
                                   'Code' : [project_code], 
                                   'Supervisor' : [project_supervisor], 
                                   'Title' : [project_title],
                                   'Subject Area' : [project_area],
                                   'Type' : [project_type],
                                   'Source' : [project_source],
                                   'Rank' : [student_rank]})
                
    df_internal_results = pd.concat([df_internal_results, student_internal_result], ignore_index=True)

    
if 'NONE' in df_internal_results.columns:
    df_internal_results.drop(columns=['NONE'], inplace=True)


print(df_internal_results)



# Use ExcelWriter to write to the specified filename
# --------------------------------------------------
with pd.ExcelWriter(filepath_internal_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    df_internal_results.to_excel(writer, sheet_name='Final Allocations', index=False)

print(f"Sheet 'Final Allocations' has been successfully written to {filepath_internal_results}")



# ------------------------------- 
# CREATE STUDENT SUMMARY SHEET 
# -------------------------------

# Create new DataFrame to save statistics as table
# -----------------------------------------------

# Define the column names
summary_columns = ['Metric', 'Count', 'Percentage']

print(summary_columns)

# Create an empty DataFrame with specified columns
summary_df = pd.DataFrame(columns=summary_columns)


# Compute student statistics data
# -------------------------------

statistics_choices = ['% 1st choice', '% 2nd choice', 
              '% 3nd choice', '% 4th choice', 
              '% 5th choice','% 6th choice', 
              '% 7th choice', '% 8th choice']

count_choices = [0, 0, 0, 0, 0, 0, 0, 0]
percentage_choices = [0, 0, 0, 0, 0, 0, 0, 0]

for index, row in df_internal_results.iterrows():
    student_rank = row['Rank']
    count_choices[student_rank-1] += 1 
    
percentage_choices = [float(count)/float(sum(count_choices)) for count in count_choices]

print('counts_choices', count_choices)
print('percentages_choices', percentage_choices)


# Save student statistics data in DataFrame 
# -----------------------------------------

summary_row = pd.DataFrame({'Metric' : statistics_choices,
                            'Count' : count_choices,
                            'Percentage' : percentage_choices})
            
summary_df = pd.concat([summary_df, summary_row], ignore_index=True)


# Use ExcelWriter to write to the specified filename
with pd.ExcelWriter(filepath_internal_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    summary_df.to_excel(writer, sheet_name='Student Allocation Statistics', index=False)

print(f"Sheet 'Student Summary Statistics' has been successfully written to {filepath_internal_results}")




# ---------------------------------
# CREATE SUPERVISOR POPULARITY SHEET
# ---------------------------------

# Compute supervisor statistics data
# ----------------------------------
'''
# Get list of supervisors:
supervisors = []

for index, row in data_topics.iterrows():
    supervisor = row['Supervisor']
    
    if supervisor not in supervisors:
        supervisors.append(supervisor)    

for index, row in df_selfproposals.iterrows():
    supervisor = row['Supervisor']

    if supervisor not in supervisors:
        supervisors.append(supervisor)
        
print(supervisors)
'''

# Get allocation count for each supervisor:
n_supervisors = len(df_supervisors) - 1

count_allocation = [0 for i in range(n_supervisors)]
count_selfproposed = [0 for i in range(n_supervisors)]
print(len(count_selfproposed))


for index, row in df_internal_results.iterrows():
    supervisor_name = row['Supervisor']
    source = row['Source']

    indices = df_supervisors.index[df_supervisors["Supervisor"] == supervisor_name]

    if indices.empty:
        print(f"Supervisor {supervisor_name} does not exist")
        continue
    if source == 'Directory':
        count_allocation[indices[0]] += 1
    else: #'Self-proposed'
        count_selfproposed[indices[0]] += 1

count_supervision = [a + b for a, b in zip(count_allocation, count_selfproposed)]
print(count_allocation)
print(count_selfproposed)
print(count_supervision)


# Get selection count for each supervisor: 
count_selections5 = [0 for i in range(n_supervisors)]
count_selections3 = [0 for i in range(n_supervisors)]

for index, row in df_students_preferences.iterrows():
    if row['Self-proposal'] == 'No':

        topics5 = [row['Topic1'], row['Topic2'], row['Topic3'], row['Topic4'], row['Topic5']]
        topics_supervisors_indices_5 = []
        topics_supervisors_indices_3 = []

        for i, topic in enumerate(topics5):
            #print(f"Topic {i}: {topic}")
            supervisor_matches = df_topics.loc[df_topics['Code'] == topic, "Supervisor"]
            if supervisor_matches.empty:
                continue           
            supervisor_name = supervisor_matches.iloc[0]
            #print(supervisor_name)

            indices = df_supervisors.index[df_supervisors["Supervisor"] == supervisor_name]
            if i < 3:
                topics_supervisors_indices_5.append(indices[0])
                topics_supervisors_indices_3.append(indices[0])
            else: 
                topics_supervisors_indices_5.append(indices[0])

            '''        
            for index2, row2 in topic_row.iterrows(): 
                supervisor = row2['Supervisor']               
                index = supervisors.index(supervisor)

                count_selections5[index] += 1 
            '''
        print(topics_supervisors_indices_5)
        topics_supervisors_indices_5_set = set(topics_supervisors_indices_5)
        print(topics_supervisors_indices_5_set)
        topics_supervisors_indices_3_set = set(topics_supervisors_indices_3)

        for index in topics_supervisors_indices_5_set:
            count_selections5[index] += 1 
        for index in topics_supervisors_indices_3_set:
            count_selections3[index] += 1

        ''' 
        for topic in topics3:
            topic_row = df_internal_results.loc[df_internal_results['Code'] == topic]
                        
            for index2, row2 in topic_row.iterrows(): 
                supervisor = row2['Supervisor']               
                index = supervisors.index(supervisor)
                count_selections3[index] += 1 
        '''

for i, row in df_selfproposals.iterrows():
    supervisor_name = row['Supervisor']
    indices = df_supervisors.index[df_supervisors["Supervisor"] == supervisor_name]
    count_selections5[indices[0]] += 1 
    count_selections3[indices[0]] += 1 
                
print(count_selections5)    
print(count_selections3) 


# Computer percentage success for each supervisor:
percentage_success5 = [0 for i in range(n_supervisors)]
percentage_success3 = [0 for i in range(n_supervisors)]

for i in range(len(percentage_success5)):
    if count_selections5[i] == 0:
        percentage_success5[i] = 1.0
    else:
        percentage_success5[i] = float(count_supervision[i])/float(count_selections5[i])
    
    if count_selections3[i] == 0:
        percentage_success3[i] = 1.0
    else:
        percentage_success3[i] = float(count_supervision[i])/float(count_selections3[i])

print(percentage_success5)
print(percentage_success3)
    

# Create new DataFrame to save statistics as table
# ------------------------------------------------
# Define the column names
summary_columns = ['Supervisor', 'Count Allocated', 'Count Selfproposed', 'Count Supervised',
                   'Count Selected (top 5)', 'Percentage Success (top 5)',
                   'Count Selected (top 3)', 'Percentage Success (top 3)']
                                
print(summary_columns)

# Create an empty DataFrame with specified columns
summary_df = pd.DataFrame(columns=summary_columns)

# Add data
for index, row in df_supervisors.iterrows():
    
    supervisor_name = row['Supervisor']

    if supervisor_name == "TOPIC AREA":
        continue

    summary_row = pd.DataFrame({'Supervisor' : [supervisor_name],
                                'Count Allocated' : count_allocation[index],
                                'Count Selfproposed' : count_selfproposed[index],
                                'Count Supervised' : count_supervision[index],
                                'Count Selected (top 5)': count_selections5[index],
                                'Percentage Success (top 5)' : percentage_success5[index],
                                'Count Selected (top 3)': count_selections3[index],
                                'Percentage Success (top 3)' : percentage_success3[index]})
        
    summary_df = pd.concat([summary_df, summary_row], ignore_index=True)


# Use ExcelWriter to write to the specified filename
# -------------------------------------------------
with pd.ExcelWriter(filepath_internal_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    summary_df.to_excel(writer, sheet_name='Supervisor Popularity', index=False)

print(f"Sheet 'Supervisor Popularity' has been successfully written to {filepath_internal_results}")

                                    
                                     
# ---------------------------------
# CREATE SUPERVISOR SUMMARY SHEET 1
# ---------------------------------

# Create new DataFrame to save statistics as table
# ------------------------------------------------
# Define the column names
summary_columns = ['Metric', 'Count', 'Percentage']

print(summary_columns)

# Create an empty DataFrame with specified columns
summary_df = pd.DataFrame(columns=summary_columns)


# Compute supervisor statistics data
# ----------------------------------
statistics_students = ['N. staff with 4 students', 'N. staff with 3 students', 
              'N. staff with 2 students', 'N. staff with 1 student', 
              'N. staff with 0 student']

count_students = [0, 0, 0, 0, 0]
percentage_students = [0, 0, 0, 0, 0]

project_counts = {}
supervisor_counts = {}

for index, row in df_topics.iterrows():
    project = row['Code']
    supervisor = row['Supervisor']
    if project not in project_counts:
        project_counts[project] = [0, supervisor]
    if supervisor not in supervisor_counts:
        supervisor_counts[supervisor] = 0

for index, row in df_selfproposals.iterrows():
    project = row['Code']
    supervisor = row['Supervisor']
    if project not in project_counts:
        project_counts[project] = [0, supervisor]
    if supervisor not in supervisor_counts:
        supervisor_counts[supervisor] = 0
        

for index, row in df_internal_results.iterrows():
    project = row['Code']    
    project_counts[project][0] += 1

print (project_counts)

for project, data in project_counts.items():
    supervisor = data[1]
    supervisor_counts[supervisor] += data[0]

print (supervisor_counts)

for supervisor, count in supervisor_counts.items():
    count_students[4-count] += 1
    
percentage_students = [float(count)/float(sum(count_students)) for count in count_students]

print('counts_students', count_students)
print('percentages_students', percentage_students)


# Save student statistics data
# -------------------------------
# Save in DataFrame
summary_row = pd.DataFrame({'Metric' : statistics_students,
                            'Count' : count_students,
                            'Percentage' : percentage_students})
            
summary_df = pd.concat([summary_df, summary_row], ignore_index=True)

# Use ExcelWriter to write to the specified filename
with pd.ExcelWriter(filepath_internal_results, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    summary_df.to_excel(writer, sheet_name='Supervisor Stats', index=False)

print(f"Sheet 'Supervisor Stats' has been successfully written to {filepath_internal_results}")

# Remove 'Sheet' Sheet
wb = openpyxl.load_workbook(filepath_internal_results)
print(wb.sheetnames)
defaultSheet = wb['Sheet']
wb.remove(defaultSheet)
wb.save(filepath_internal_results)
